In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
eval_iters = 250
learning_rate = 3e-4


cuda


In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocabulary_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [3]:

string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([ 1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,
         0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,
         0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36,
        25, 38, 28,  1, 39, 30,  1, 39, 50,  9])


In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x, y
x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[58, 75, 58, 67,  1, 54,  0, 73],
        [56, 61,  1, 54, 72,  1, 78, 68],
        [58,  1, 72, 54, 62, 57,  9,  1],
        [69, 58, 56, 74, 65, 62, 54, 71]], device='cuda:0')
targets:
tensor([[75, 58, 67,  1, 54,  0, 73, 61],
        [61,  1, 54, 72,  1, 78, 68, 74],
        [ 1, 72, 54, 62, 57,  9,  1,  3],
        [58, 56, 74, 65, 62, 54, 71,  1]], device='cuda:0')


In [5]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is ', context, 'target is ', target)

when input is  tensor([1]) target is  tensor(1)
when input is  tensor([1, 1]) target is  tensor(28)
when input is  tensor([ 1,  1, 28]) target is  tensor(39)
when input is  tensor([ 1,  1, 28, 39]) target is  tensor(42)
when input is  tensor([ 1,  1, 28, 39, 42]) target is  tensor(39)
when input is  tensor([ 1,  1, 28, 39, 42, 39]) target is  tensor(44)
when input is  tensor([ 1,  1, 28, 39, 42, 39, 44]) target is  tensor(32)
when input is  tensor([ 1,  1, 28, 39, 42, 39, 44, 32]) target is  tensor(49)


In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [8]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocabulary_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


PZ)p02ZW2q["_!80kZWs]PDWJ;UEMe-Yp'd,y081G)94uO,j3&K5]PPD_ZtK)lEv_tcC8GIiip]*:T27XaP4'LI4tN63"?pVlvmUi22.p]m(;x.dktUd9KNC3oxGNk6*&
*!p3l8SCCdX*rnJx?IdX*xCH:v0]iMe';00WQw,-0d,l[aBjm-pOZk;q!Qdmbkb8,sN._fO"uI)( ,a8kh*R1,nyYp XwquVwtJJ[ao3,P6O"Adbxp7C?CMv n3T,0zwTs!tXWgqdd2gRf z'zr;bz'r7C;VVKSEc45-po(J;VY_4PTYzsG.H9e?:'TfJ,nUdXbzBRwyv-?&SEcKU!uHea?eeIaY*-EyU0d8z'LbcWYO"P6KBQc4 3jEhEm?w]PP3,H"tbKG"H;ext6d7BOKro4-RTs'MeSjEI4n5_7wdu)p UK'rYCMqlmUqs3mcszZ(V7.*Udy!tKUGJ,RSwxOSKxpyDobSc)LI6* wIcc4;((;m3drz


In [9]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.899, val loss: 4.855
step: 250, train loss: 4.821, val loss: 4.801
step: 500, train loss: 4.749, val loss: 4.744
step: 750, train loss: 4.673, val loss: 4.682
4.5435333251953125
